Contains information from [Kaggle](https://www.kaggle.com/soroushghaderi/chocolate-bar-2020) which is made available
under the ODC Attribution License.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection    import train_test_split, RandomizedSearchCV
from sklearn.linear_model       import Ridge
from sklearn.ensemble           import RandomForestRegressor
from sklearn.impute             import MissingIndicator, SimpleImputer
from sklearn.metrics            import r2_score
from sklearn.pipeline           import Pipeline
from sklearn.preprocessing      import StandardScaler, OneHotEncoder
from sklearn.compose            import ColumnTransformer
from sklearn.feature_selection  import SelectFromModel
from sklearn.inspection         import permutation_importance

In [3]:
chocolate_df = pd.read_csv('chocolate.csv', index_col='Unnamed: 0')

In [4]:
# 'ref' is the id of the taste test dropping it to avoid data leak
X = chocolate_df.drop(['ref', 'rating'], axis=1)
y = chocolate_df['rating']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

# Checking for missing values

In [25]:
# Checking for missing data
indicator = MissingIndicator(features="all")
missing = indicator.fit_transform(X_train.values)
# summing column wise to do a count of missing values for each columns
missing.sum(axis=0)

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   52,  452, 1482])

So only the last 3 columns has missing values which are 2nd to 4th taste test. They are all categorical variables just imputing it as 'missing' because these taste test are optional.

In [7]:
indicator = MissingIndicator(features="all")
missing = indicator.fit_transform(y_train.values.reshape(-1, 1))
missing.sum(axis=0)

array([0])

there's no missing values in y

# Model / Feature / Hyperparmeter Selection
I am mainly interested in how `Ridge` and `RandomForestRegressor` differ in their performance on this data set. So I will be focusing on comparing these two models.

In [8]:
# preprocessing pipeline set up
categorical_columns = (X_train.dtypes == object)
# treating 'review_date' as categorical too because it's the year
categorical_columns['review_date'] = True

# the preprocessing code used here are inspired by codes from class
con_pipe = Pipeline(
    [
        ('scaler', StandardScaler())
    ]
)

cat_pipe = Pipeline(
    [
        (
            'imputer',
            SimpleImputer(
                strategy='constant', fill_value='missing', add_indicator=True,
            )
        ),
        (
            'ohe',
            OneHotEncoder(handle_unknown='ignore')
        ),
    ]
)

preprocessing = ColumnTransformer(
    [
        ('categorical', cat_pipe,  categorical_columns),
        ('continuous',  con_pipe, ~categorical_columns),
    ]
)

In [11]:
# ridge
search_space = {
    # alpha is the penalty rate which determines
    # how much we are restrciting the coeffcients by 
    'ridge__alpha': np.logspace(0, 4, 10),
}

ridge_pipe = Pipeline(
    [
        ('preprocessing', preprocessing),
        # log2 to reduce run time, otherwise it takes too long
        # using select from model to do automatic feature selection
        ('sfm', SelectFromModel(
                RandomForestRegressor(max_features="log2"),
                # choosing 450 because it was a good balance point
                # from my initial testing. Didn't put it as a hyper param
                # here because it would take too long
                max_features=450,
            ),
        ),
        ('ridge', Ridge()),
    ]
)

ridge_rscv = RandomizedSearchCV(
    ridge_pipe,
    search_space,
    n_jobs=-1,
)
ridge_rscv.fit(X_train, y_train)

clean_ridge_best_param = {
    k.replace('ridge__', ''): v
    for k, v in ridge_rscv.best_params_.items()   
}

print(clean_ridge_best_param)
print('R^2: ', ridge_rscv.best_score_)

{'alpha': 2.7825594022071245}
R^2:  0.3024424589293524


In [9]:
# random forest regressor
# the structure of this basically repeats the previous cell
# excep the search space
search_space = {
    # testing out n_estimators because too few trees could lead to a bad model
    # but too many will also slow down the fitting
    'rfr__n_estimators': [20, 50, 100, 150, 200],
    # avoid each tree from becoming too specific,
    # but also to reduce run time when fitting
    'rfr__max_depth': range(1, 20, 2),
    # avoid each tree from becoming too specific,
    # but also to reduce run time when fitting
    'rfr__min_samples_split': range(2, 10),
    # limiting max feature help make the trees more independent
    'rfr__max_features': ["sqrt", "log2"],
    # avoid each tree from becoming too specific,
    # but also to reduce run time when fitting
    'rfr__min_samples_leaf': range(1, 50, 5),
    # avoid each tree from becoming too specific,
    # but also to reduce run time when fitting
    'rfr__max_leaf_nodes': range(1, 100, 5),
    # help trees become more independent by limiting the
    # amount of data they see
    'rfr__max_samples': [0.6, 0.8, 1],
}

rfr_pipe = Pipeline(
    [
        ('preprocessing', preprocessing),
        ('sfm', SelectFromModel(
                RandomForestRegressor(max_features="log2"),
                max_features=450,
            )
        ),
        ('rfr', RandomForestRegressor()),
    ]
)

rfr_rscv = RandomizedSearchCV(
    rfr_pipe,
    search_space,
    n_jobs=-1,
)
rfr_rscv.fit(X_train, y_train)

clean_rfr_best_param = {
    k.replace('rfr__', ''): v
    for k, v in rfr_rscv.best_params_.items()
}

print(clean_rfr_best_param)
print('R^2: ', rfr_rscv.best_score_)

{'n_estimators': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': 0.8, 'max_leaf_nodes': 41, 'max_features': 'log2', 'max_depth': 13}
R^2:  0.10775484982259846


My final model choice is `Ridge` since it did much better than random forest regressor in this case.

The model is `Ridge(alpha=2.7825594022071245)`

# Final Model

In [12]:
# fit final model
sfm = ridge_rscv.best_estimator_.named_steps['sfm']

final_ridge_pipe = Pipeline(
    [
        ('preprocessing', preprocessing),
        ('sfm', sfm),
        ('ridge', Ridge(alpha=2.7825594022071245)),
    ]
)
final_ridge_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  company                              True
company_location                     True
review_date                          True
country_of_bean_origin               True
specific_bean_origin_or_bar_name     True
cocoa_perc...
cocoa_percent                        True
counts_of_ingre

In [23]:
# model performance on test data
# using R2 to measure the percentage of variance in the target explained by the model
# because using MAE we can compare how two model is performing but harder to tell how 
# one model is performing
print('R2 score: ', final_ridge_pipe.score(X_test, y_test))

R2 score:  0.3406171493333361


# Conclusion

In [ ]:
# retrieving the name of the feature selected
categorical_feature_names = final_ridge_pipe['preprocessing'].named_transformers_['categorical']['ohe'].get_feature_names().tolist()
feature_names = np.array(categorical_feature_names + list(categorical_columns[categorical_columns==False].index))

feature_idx = final_ridge_pipe['sfm'].get_support()
selected_features = feature_names[feature_idx]

In [ ]:
# checking which variables are used
variable_index = set(int(i.split('_')[0].replace('x', '')) for i in selected_features[:-8])
print('Params used in model: ')
for idx in variable_index:
    name = categorical_columns[categorical_columns].index[idx]
    print('CAT - ', idx, '-', name)
for feature in selected_features[-2:]:
    print('CON - ', feature)

Params used in model: 
CAT -  0 - company
CAT -  1 - company_location
CAT -  2 - review_date
CAT -  3 - country_of_bean_origin
CAT -  4 - specific_bean_origin_or_bar_name
CAT -  6 - cocoa_butter
CAT -  7 - vanilla
CAT -  8 - lecithin
CAT -  9 - salt
CAT -  10 - sugar
CAT -  11 - sweetener_without_sugar
CAT -  12 - first_taste
CAT -  13 - second_taste
CAT -  14 - third_taste
CAT -  15 - fourth_taste
CON -  cocoa_percent
CON -  counts_of_ingredients


In [ ]:
# checking the top 10 biggest coeeficient
sorted(zip(selected_features, final_ridge_pipe['ridge'].coef_), key=lambda x: -np.abs(x[1]))[:10]

[('x14_very bitter', -0.4956080384492338),
 ('x13_medicinal', -0.4230425478907575),
 ('x12_pastey', -0.3651514109881082),
 ('x4_Dark', -0.3647776843397996),
 ('x0_Madecasse (Cinagra)', 0.35825182510445275),
 ('x13_pungent', -0.32992480474533176),
 ('x12_burnt rubber', -0.31631653286007416),
 ('x13_strong chemical', -0.31442311268972994),
 ('x12_perfume', -0.30604120029818793),
 ('x13_dirty', -0.3019164666372393)]


The top 10 coefficients are mostly from the taste test which makes sense since the rating is an objective score. It it very interesting that they can taste `burnt rubber`, `dirty`, `pungent` from chocolate and reasonably these taste impacted the rating negatively. `x0_Madecasse (Cinagra)` is the name of a company and the chocolate being that brand impacts the score positively, so perhaps it's a chocolate brand to one might want to try out.

Based on the coefficients, it seems like taste test are very directly correlated with the score that the biggest coefficient does not involve other features of chocolate (e.g. kind of beans, cocoa content). One challenge of this data set is that many of the columns contains data that holds a lot of meaning (e.g. taste test) that one hot encoding is not able to capture, hence we only have a R^2 score of 0.3. An interesting next step another step would be to use custom word embedding to better encode this dataset to study the relationship between features of chocolate and the flavor graders taste and hence the rating.

This information could potentially be useful in reducing the cost to develop new and tasty chocolates, by simply predicting the rating of the chocolate from the ingredients they are using. It could also benefits consumers in a way where we can avoid bad tasting chocolate with taste like `burnt rubber` and `dirty` and save our money for amazing chocolates.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=15c88ccc-0e75-46fc-8561-25080a52c07c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>